In [1]:
try:
    import anastruct, openturns, numba, joblib
except:
    import os
    if os.sys.platform == 'linux' :
        file_path = 'sensitivityEnv.yml'
        os.system('conda env create -f'+file_path)
        # to have the right modules installed
        print('now activate the environment and restart jupyter with other kernel')
    else :
        print('Do it alone')

In [2]:
import NdGaussianProcessSensitivity as ngps
import NdGaussianProcessConstructor as ngpc
# Classes utilitaires
import numpy                        as np
import openturns                    as ot
import matplotlib.pyplot            as plt
from   importlib                import reload 

# on importe aussi les fonctions à étudier
import RandomBeamGenerationClass    as rbgc

trend function args:  ['x']  trend function:  210000 

Please be aware that the number of elements in the argument list has to be the same as the dimension of the process:  1
trend function args:  ['x']  trend function:  10 

Please be aware that the number of elements in the argument list has to be the same as the dimension of the process:  1


In [3]:
# process governing the young modulus for each element      (MPa)
process_E = ngpc.NdGaussianProcessConstructor(dimension=1,
                                              grid_shape=[[0,1000,100],],
                                              covariance_model={'NameModel':'MaternModel',
                                                                'amplitude':5000.,
                                                                'scale':300,
                                                                'nu':13/3},
                                              trend_arguments=['x'],trend_function=210000)
process_E.setName('E_')


# process governing the diameter for each element          (mm)
process_D = ngpc.NdGaussianProcessConstructor(dimension=1,
                                              grid_shape=[[0,1000,100],],
                                              covariance_model={'NameModel':'MaternModel',
                                                                'amplitude':.3,
                                                                'scale':250,
                                                                'nu':7.4/3},
                                              trend_arguments=['x'],trend_function=10)
process_D.setName('D_')


# random variable for the density of the material (kg/m³)
rho         = 7850.
sigma       = 250
nameD       = 'Rho'
RV_Rho = ngpc.NormalDistribution(mu = rho, sigma = sigma, name = nameD)


# random variable for the position of the force   (mm) 
middle       = 500
sigma_f      = 50
namePos     = 'FP'
RV_Fpos = ngpc.NormalDistribution(mu = middle, sigma = sigma_f, name = namePos)


# random variable for the norm of the force    (N)
muForce       = 100
# we go from sigma = 15 to sigma = 1.5, as the influence is too important
sigma_Fnor    = 1.5
nameNor       = 'FN'
RV_Fnorm = ngpc.NormalDistribution(mu = muForce, sigma = sigma_Fnor, name = nameNor)

trend function args:  ['x']  trend function:  210000 

Please be aware that the number of elements in the argument list has to be the same as the dimension of the process:  1
trend function args:  ['x']  trend function:  10 

Please be aware that the number of elements in the argument list has to be the same as the dimension of the process:  1


In [4]:
from importlib import reload
reload(ngps)
outputVariables = {'out1' :
                   {
                         'name'     : 'VonMisesStress',
                         'position' : 0,
                         'shape'    : (102,)  
                    },
                   'out2' :
                   {
                        'name'      : 'maxDeflection',
                        'position'  : 1,
                        'shape'     : (1,)
                   }
                  }
#Pour utiliser notre fonction, un wrapper a été spécialement écrit pour faciliter l'accès aux fonctions mais
#ce choix est entièrement dépendant de la manière dont a été définie la fonction sur laquelle vous travaillez.
functionWrapper = rbgc.sampleAndSoloFunctionWrapper(process_E, process_D, RV_Rho, RV_Fpos, RV_Fnorm)


In [5]:
inputVarList = [process_E, process_D, RV_Rho, RV_Fpos, RV_Fnorm]
# We also the need the two functions of the model (one for samples, the other for single evaluations)
# In our case, as our model is defined as a class, we have to first create the model, 
# but it also could just be just two functions taking as an input the fields and RVs
soloFunction   = functionWrapper.randomBeamFunctionSolo
sampleFunction = functionWrapper.randomBeamFunctionSample
##
size           = 100 ## Number of samples for our sobol indicies experiment (kept low here to make things faster)
##
reload(ngps)
##
processSensitivityAnalysis = ngps.NdGaussianProcessSensitivityAnalysis(inputVarList, 
                                                                       outputVariables,
                                                                       sampleFunction,
                                                                       soloFunction,
                                                                       size)

Input Variables are (without Karhunen Loeve Decomposition) :
 ['E_', 'D_', 'Rho', 'FP', 'FN'] 

Output Variables are :
 ['VonMisesStress', 'maxDeflection'] 

trend function args:  ['x']  trend function:  210000 

Please be aware that the number of elements in the argument list has to be the same as the dimension of the process:  1
trend function args:  ['x']  trend function:  10 

Please be aware that the number of elements in the argument list has to be the same as the dimension of the process:  1


In [6]:
composedDist = processSensitivityAnalysis.wrappedFunction.KLComposedDistribution

In [7]:
composedDist.getSample(10)

class=Sample name=ComposedDistribution implementation=class=SampleImplementation name=ComposedDistribution size=10 dimension=24 description=[X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23] data=[[52.714,-0.619011,23.3475,0.0634781,17.4149,0.734408,13.4944,-1.27923,46.1193,0.707394,18.8688,-0.183863,13.4788,-0.209803,9.41822,-0.0742153,6.48239,-1.5744,7.34366,0.0329441,5.14246,8044.59,516.9,100.305],[55.6675,-1.40917,25.1367,0.102876,16.3698,1.36493,14.8984,-0.1152,46.5483,1.37919,18.6865,1.56735,13.2207,0.479475,9.11985,-0.288072,7.07406,-1.0552,7.27852,0.111951,7.01101,7361.83,557.496,99.0891],[56.3178,1.6349,24.4509,1.22769,15.9309,0.340661,14.1665,-0.107663,45.7921,-0.394271,19.6004,0.770477,12.6181,-0.755311,10.3003,-0.813007,7.56553,-0.9519,6.58463,-0.226705,6.01415,7714.68,429.043,101.168],[54.3503,0.51795,23.7728,-0.215237,18.1493,-0.108873,14.7482,-0.268838,49.143,1.51779,19.1317,-0.0921642,13.4952,1.23331,9.4259,-0.765836,9.26839,-1.79759,7.83842,-0.663389,6.82083,7674.93,461.131,97.81],[53.7058,1.35787,24.5834,0.982931,17.1453,0.167166,14.5404,-0.843198,49.3897,0.514648,17.8421,-0.868669,12.5009,0.0785301,9.18282,-0.814641,7.12096,0.35955,5.80957,0.230075,4.25693,8106.57,577.597,99.6555],[54.0285,-0.591317,25.5378,-1.01462,19.0283,0.378518,15.4357,0.673863,48.0528,-0.0365893,18.7705,-0.90374,13.9514,0.508557,10.1649,-0.590025,7.55525,-1.56937,5.66099,-0.822967,4.36744,8036.99,465.018,102.539],[55.0481,0.0174348,24.969,-0.487484,17.3591,1.2696,15.318,-0.0324894,47.4287,1.68887,19.2597,0.928307,12.1685,1.61488,10.8164,0.987393,8.33043,-0.965557,8.70749,-0.68446,6.92034,7828.16,448.581,98.6132],[55.7546,1.60473,24.0468,-1.77038,18.9616,0.350309,13.0493,-0.369532,46.3547,-0.174033,19.0093,-0.743797,13.3728,-0.521012,9.98108,-0.075688,10.0352,-0.643848,5.52265,-0.629788,4.3667,7660.37,563.575,96.5005],[54.1818,-1.33513,25.3203,0.269429,17.7215,-1.92634,14.9109,-0.668414,46.2402,1.14093,17.9661,-0.139252,11.5621,0.0117174,10.7147,-1.25323,7.1406,0.00175656,6.19896,1.67735,3.43944,7479.14,480.935,98.8917],[53.7191,-0.630469,22.5979,-0.149521,18.2351,0.793165,15.9497,-0.569378,46.1946,-0.407148,19.3767,-1.17333,14.0775,-1.63754,10.0978,0.587523,7.37506,-0.841714,8.6772,0.0936265,6.16765,7655.08,439.271,98.3837]]

In [8]:
print(processSensitivityAnalysis.wrappedFunction.inputVarNames)
print(processSensitivityAnalysis.wrappedFunction.inputVarNamesKL)
print(processSensitivityAnalysis.wrappedFunction._inputVarOrdering)

['E_', 'D_', 'Rho', 'FP', 'FN']
['E_xi_0', 'E_xi_1', 'E_xi_2', 'E_xi_3', 'E_xi_4', 'E_xi_5', 'E_xi_6', 'E_xi_7', 'D_xi_0', 'D_xi_1', 'D_xi_2', 'D_xi_3', 'D_xi_4', 'D_xi_5', 'D_xi_6', 'D_xi_7', 'D_xi_8', 'D_xi_9', 'D_xi_10', 'D_xi_11', 'D_xi_12', 'Rho', 'FP', 'FN']
['var0', 'var1', 'var2', 'var3', 'var4']


In [9]:
import customWraps as cw

In [10]:
@cw.timing
def getLHSExperiment(distribution, size, alwaysShuffle, randomShift):
    return ot.LHSExperiment(distribution, size, alwaysShuffle, randomShift).generate()

In [11]:
exp = getLHSExperiment(composedDist, 2000, True, True)

timed  0.049272776  s for function " getLHSExperiment "


In [12]:
print('That works really fast!')
exp

That works really fast!


class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=2000 dimension=24 description=[X0,X1,X2,...,X21,X22,X23] data=[[54.9172,1.73854,25.8447,...,7905.8,517.968,100.45],[52.2721,-0.591491,24.2068,...,7436.59,513.713,97.9913],[54.9694,-0.278001,22.4816,...,7712.36,445.384,99.5909],...,[54.9063,-0.331151,23.2092,...,7564.91,561.111,99.8302],[53.7533,0.187766,22.3443,...,7802.59,491.434,100.532],[54.3088,0.702265,25.1075,...,7617.01,515.873,99.231]]

In [201]:
import NdGaussianProcessExperimentGeneration as ngpeg
reload(ngpeg)

<module 'NdGaussianProcessExperimentGeneration' from '/home/motherloadubu/Documents/project_PHIMECA/stochastic_process_analysis/NdGaussianProcessExperimentGeneration.py'>

In [202]:
test = ngpeg.NdGaussianProcessExperiment()

Generation types:
1 : Random (default)
2 : LHS
3 : LowDiscrepancySequence
4 : SimulatedAnnealingLHS


In [203]:
test.setSampleSize(100)

In [204]:
test.setOTPyFunctionWrapper(processSensitivityAnalysis.wrappedFunction)

In [205]:
test.setGenType(1)
test.generateSample()

In [206]:
test.setGenType(2)
test.generateSample()

In [207]:
test.setGenType(3)
test.generateSample(sequence='Halton')

In [208]:
test.setGenType(4)
arr=test.generateSample()

undefined parameter 'SpaceFilling', setting to default 'SpaceFillingC2'
undefined parameter 'TemperatureProfile', setting default GeometricProfile


In [209]:
print(test.inputVarNames)
print(test.inputVarNamesKL)
test.getDataFieldAndRV()

['E_', 'D_', 'Rho', 'FP', 'FN']
['E_xi_0', 'E_xi_1', 'E_xi_2', 'E_xi_3', 'E_xi_4', 'E_xi_5', 'E_xi_6', 'E_xi_7', 'D_xi_0', 'D_xi_1', 'D_xi_2', 'D_xi_3', 'D_xi_4', 'D_xi_5', 'D_xi_6', 'D_xi_7', 'D_xi_8', 'D_xi_9', 'D_xi_10', 'D_xi_11', 'D_xi_12', 'Rho', 'FP', 'FN']
i= 0 k= 1 jump= 0
i= 0 k= 2 jump= 0
i= 0 k= 3 jump= 0
i= 0 k= 4 jump= 0
i= 0 k= 5 jump= 0
i= 0 k= 6 jump= 0
i= 0 k= 7 jump= 0
i= 0 k= 8 jump= 0
i= 1 k= 1 jump= 7
i= 1 k= 2 jump= 7
i= 1 k= 3 jump= 7
i= 1 k= 4 jump= 7
i= 1 k= 5 jump= 7
i= 1 k= 6 jump= 7
i= 1 k= 7 jump= 7
i= 1 k= 8 jump= 7
i= 1 k= 9 jump= 7
i= 1 k= 10 jump= 7
i= 1 k= 11 jump= 7
i= 1 k= 12 jump= 7
i= 1 k= 13 jump= 7
i= 2 k= 1 jump= 19
i= 3 k= 1 jump= 19
i= 4 k= 1 jump= 19


In [210]:
test.dataMixSamples

[8, 13, 1, 1, 1]

In [211]:
test.sample_A.shape

(100, 24)

In [212]:
test.getExperiment()

In [213]:
test.experimentSample.shape

(700, 24)

In [214]:
np.unique(np.argwhere(test.experimentSample[200:300,...]==test.sample_B)[...,1])

array([0, 1, 2, 3, 4, 5, 6, 7])

In [215]:
np.unique(np.argwhere(test.experimentSample[300:400,...]==test.sample_B)[...,1])

array([ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

In [216]:
np.unique(np.argwhere(test.experimentSample[400:500,...]==test.sample_B)[...,1])

array([21])

In [217]:
np.unique(np.argwhere(test.experimentSample[500:600,...]==test.sample_B)[...,1])

array([22])

In [218]:
np.unique(np.argwhere(test.experimentSample[600:700,...]==test.sample_B)[...,1])

array([23])